In [1]:
import pandas as pd
import numpy as np
import sklearn
from xgboost import XGBClassifier
import xgboost
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from xgboost.sklearn import XGBClassifier
from sklearn import metrics   #Additional scklearn functions
from sklearn.model_selection import cross_validate,GridSearchCV

import pickle
from sklearn.utils import shuffle
import json
import os
from xgboost import XGBClassifier
from matplotlib import pyplot
import matplotlib.pylab as plt
%matplotlib inline
import math

In [ ]:
#Don't use datetime, Don't use categorical features, xgboost doesn't do those, first one hot encode
featuresToUse = ['visitor_hist_starrating', 'visitor_hist_adr_usd',
       'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'comp1_rate', 'comp1_inv',
                 'mean_prop_starrating',
       'mean_prop_review_score', 'mean_prop_brand_bool',
       'mean_prop_location_score1', 'mean_prop_location_score2',   'mean_prop_log_historical_price','cluster__0','cluster__1','cluster__2','cluster__3','cluster__4','cluster__5','diff_distance_orig','diff_price_usd']
labelToPredict = 'booking_bool'
neededVars = ['srch_id','click_bool']
nameOfModel = "models/10mayClickingOverFitTest"

In [ ]:
dftrain = pd.read_csv('../data/80PercentTrainWithMeanNoShuffle.csv', delimiter=",",usecols=featuresToUse+[labelToPredict]+neededVars)
dftrain = dftrain.sample(frac=0.2)
Y = dftrain[labelToPredict]

In [ ]:
 'min_child_weight':range(1,12,2),
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)],
 'gamma':[i/10.0 for i in range(0,5)],
 'learning_rate':[i/10.0 for i in range(0,10)]

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
param_test1 = {
 'max_depth':range(3,5,2),

}



gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate = 0.1, n_estimators=120, max_depth=5,
min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
objective= 'reg:logistic', nthread=-1, scale_pos_weight=sum(Y==0)/sum(Y==1), seed=12,verbosity=3), 
param_grid = param_test1, scoring='roc_auc',n_jobs=5,iid=False, cv=5,verbose = 100)

gsearch1.fit(dftrain[featuresToUse],dftrain[labelToPredict])
gsearch1.cv_results_

In [ ]:
gsearch1.cv_results_

In [ ]:
file = open("cvScores.txt")
 
file.write(str(gsearch1.cv_results_))
 
file.close() 